<a href="https://colab.research.google.com/github/Rominaarjomand/ML-projects/blob/master/PredectionOfOnlineRetailShop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"rominaarjomand","key":"057a3141cec5b6a0c8103c54a951f8e8"}'}

In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [9]:
!pip install kaggle


In [10]:
!kaggle datasets download -d ulrikthygepedersen/online-retail-dataset

Dataset URL: https://www.kaggle.com/datasets/ulrikthygepedersen/online-retail-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
online-retail-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
!unzip online-retail-dataset.zip

Archive:  online-retail-dataset.zip
replace online_retail.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [99]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches

In [100]:
orginal_df = pd.read_csv("online_retail.csv")
orginal_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [101]:
df = orginal_df[["StockCode","Description","CustomerID"]]
df

,StockCode,Description,CustomerID
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,17850.0
1,71053,WHITE METAL LANTERN,17850.0
2,84406B,CREAM CUPID HEARTS COAT HANGER,17850.0
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,17850.0
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,17850.0
...,...,...,...
541904,22613,PACK OF 20 SPACEBOY NAPKINS,12680.0
541905,22899,CHILDREN'S APRON DOLLY GIRL,12680.0
541906,23254,CHILDRENS CUTLERY DOLLY GIRL,12680.0
541907,23255,CHILDRENS CUTLERY CIRCUS PARADE,12680.0


In [102]:
for col in ["Description" , "StockCode"]:
  df[col] = df[col].str.replace(" ","_")


df["MixedFeatures"]= df["Description"] +" "+ df["StockCode"]
df = df.drop_duplicates(subset=["Description"]).reset_index(drop=True)

df

/tmp/ipython-input-2969104519.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace(" ","_")
/tmp/ipython-input-2969104519.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MixedFeatures"]= df["Description"] +" "+ df["StockCode"]


,StockCode,Description,CustomerID,MixedFeatures
0,85123A,WHITE_HANGING_HEART_T-LIGHT_HOLDER,17850.0,WHITE_HANGING_HEART_T-LIGHT_HOLDER 85123A
1,71053,WHITE_METAL_LANTERN,17850.0,WHITE_METAL_LANTERN 71053
2,84406B,CREAM_CUPID_HEARTS_COAT_HANGER,17850.0,CREAM_CUPID_HEARTS_COAT_HANGER 84406B
3,84029G,KNITTED_UNION_FLAG_HOT_WATER_BOTTLE,17850.0,KNITTED_UNION_FLAG_HOT_WATER_BOTTLE 84029G
4,84029E,RED_WOOLLY_HOTTIE_WHITE_HEART.,17850.0,RED_WOOLLY_HOTTIE_WHITE_HEART. 84029E
...,...,...,...,...
4219,85104,????damages????,NaN,????damages???? 85104
4220,21693,mixed_up,NaN,mixed_up 21693
4221,22578,lost,NaN,lost 22578
4222,85123A,CREAM_HANGING_HEART_T-LIGHT_HOLDER,17841.0,CREAM_HANGING_HEART_T-LIGHT_HOLDER 85123A


In [109]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer(stop_words="english")

df = df.dropna(subset=["MixedFeatures"])
vectorr = vectorizer.fit_transform(df["MixedFeatures"])
vectorizer.vocabulary_
similarities = cosine_similarity(vectorr)
similarities

array([[1.        , 0.        , 0.        , ..., 0.        , 0.66666667,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.66666667, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [140]:
product = input("Enter product that you have bought: ")
# product = "WHITE_HANGING_HEART_T-LIGHT_HOLDER"

try:
  product_index= df.where(df["Description"] == product).dropna().index[0]

  product_df = np.c_[df['Description'], similarities]

  similar_products = list(np.argsort(similarities[product_index]))

  similar_products.reverse()
  n_recommendation = 10
  recommended_index= [int(i) for i in similar_products[1:n_recommendation]]

  # print(df["Description"].iloc[recommended_index])

  for i in recommended_index:
      print(f"{df['Description'].iloc[i]}  |  similarity: {similarities[product_index][i]:.3f}")

except:
    print("Product is not in my list")

Enter product that you have bought: WHITE_HANGING_HEART_T-LIGHT_HOLDER
CREAM_HANGING_HEART_T-LIGHT_HOLDER  |  similarity: 0.667
wrongly_marked_carton_22804  |  similarity: 0.408
EAU_DE_NILE_JEWELLED_T-LIGHT_HOLDER  |  similarity: 0.333
HANGING_HEART_ZINC_T-LIGHT_HOLDER  |  similarity: 0.333
GOLD_SCROLL_GLASS_T-LIGHT_HOLDER  |  similarity: 0.333
RIDGED_GLASS_T-LIGHT_HOLDER  |  similarity: 0.333
MINI_CAKE_STAND_T-LIGHT_HOLDER  |  similarity: 0.333
FLOWERS_CHANDELIER_T-LIGHT_HOLDER  |  similarity: 0.333
IVORY_CHANDELIER_T-LIGHT_HOLDER  |  similarity: 0.333
